In [1]:
import io
import stardog
import pandas as pd
import seaborn as sns

In [2]:
connection_details = {
  'endpoint': 'http://localhost:5820',
  'username': 'admin',
  'password': 'admin'
}

In [3]:
database_name = 'VICODI'

with stardog.Admin(**connection_details) as admin:
    if database_name in [db.name for db in admin.databases()]:
        admin.database(database_name).drop()
    db = admin.new_database(database_name)

In [4]:
conn = stardog.Connection(database_name, **connection_details)
conn.begin()

'e8b6cf75-e5d2-42f3-bd77-71b75eeba821'

In [5]:
conn.add(stardog.content.File('../../ontologies/vicodi_all.owl'), graph_uri="http://vicodi.org/ontology")
conn.commit()

In [6]:
if conn.is_consistent(graph_uri="http://vicodi.org/ontology"):
    print("The data is consitent")
else:
    print("The data is NOT consitent")

StardogException: [500] 000012: Unrecognized schema: CONSISTENT

In [5]:
query = """
 PREFIX : <http://vicodi.org/ontology#>
            PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
            SELECT ?s
            WHERE {
                :Professor rdfs:subClassOf ?s .
            }
"""

csv_results = conn.select(query, content_type='text/csv', reasoning=True)
df = pd.read_csv(io.BytesIO(csv_results))
print(df)

Empty DataFrame
Columns: [s]
Index: []


In [23]:
query = """
 PREFIX : <http://vicodi.org/ontology#>
            SELECT ?s
            WHERE {
                ?s a :Individual .
                ?s :hasRole ?r .
                ?r a :Professor .
            }
"""

csv_results = conn.select(query, content_type='text/csv', reasoning=True)
df = pd.read_csv(io.BytesIO(csv_results))
print(df)

Empty DataFrame
Columns: [s]
Index: []


In [24]:
conn.__exit__()